In [2]:
from shapely import wkt
from shapely.geometry import Point
import geojson
import pandas as pd
import numpy as np
import keplergl
from odps.df import DataFrame
from util import GisTransform
import json
from shapely.geometry import MultiPoint, Polygon, MultiPolygon, Point
import geopandas
import hashlib

gis = GisTransform('wgs84', 'gcj02')

In [14]:
partial = pd.read_csv('dd_spider.csv')
df = pd.read_csv('temp_dw_ai_normal_deliver_range_aoi_basic_shaoxing_20201011_1603284421564.txt')

In [5]:
# partial = partial.dropna()

In [15]:
m = pd.merge(partial, df[['id', 'shape', 'name']], on='id', how='inner')

In [16]:
pd.merge(partial, df[['id', 'name']], on='id', how='inner').to_csv('report.csv', index=False)

In [17]:
def transfer_to_geojson(poly):
    if isinstance(poly, float):
        return None
    
    points = []
    for item in poly.split(';'):
        arr = item.split(',')
        pair = gis.gcj02_to_wgs84(float(arr[0]), float(arr[1]))
        points.append([pair[0], pair[1]])

    polygon = {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [points]
      },
      "properties": {
        "name": "shop delivery area"
      }
    }
    
    return polygon

In [18]:
m['geojson'] = m['shape'].apply(lambda item: transfer_to_geojson(item))

In [19]:
m[m['is_dispatch']==1].iloc[0]['geojson']

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[120.50551820057288, 30.041364771203234],
    [120.50575009131082, 30.041208458122718],
    [120.5058173463996, 30.04120265695768],
    [120.50614960542173, 30.041179638659454],
    [120.50622790259276, 30.04116586992741],
    [120.50693150133729, 30.041904938991593],
    [120.5067969852669, 30.042011541825236],
    [120.50690538462399, 30.042133860264773],
    [120.50648678416997, 30.04238562431396],
    [120.50551820057288, 30.041364771203234]]]},
 'properties': {'name': 'shop delivery area'}}

In [20]:
m.head()

,id,geohash,is_dispatch,station_id,shape,name,geojson
0,B0FFH1NC1J,wtmdyh,1,5ce7760e25e1f7d35e8b456a,"120.509794,30.038877;120.510025,30.03872;120.5...",梅和家园,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
1,B0FFG0WSHU,wtmdyh,1,5ce7760e25e1f7d35e8b456a,"120.505462,30.034673;120.506363,30.035623;120....",澄海嘉园,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
2,B023F0PO7J,wtmdyh,1,5ce7760e25e1f7d35e8b456a,"120.509203,30.03662;120.509778,30.037162;120.5...",西泽经济合作社工业园区,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
3,B0FFGIWH7O,wtmdyh,1,5ce7760e25e1f7d35e8b456a,"120.505797,30.038429;120.505669,30.03851;120.5...",浙江富美园,"{'type': 'Feature', 'geometry': {'type': 'Poly..."
4,B023F0Q9NQ,wtmdyh,1,5ce7760e25e1f7d35e8b456a,"120.503386,30.033982;120.504205,30.034769;120....",澄海小学,"{'type': 'Feature', 'geometry': {'type': 'Poly..."


In [21]:
amap = keplergl.KeplerGl(height=800)  

stations = m['station_id'].drop_duplicates().values
for station_id in stations:
    amap.add_data(data=m[m['station_id']==station_id], name=station_id)
    
amap

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'5ce7760e25e1f7d35e8b456a': {'index': [0, 1, 2, 3, 4, 5, 268, 269, 270, 271, 272, 273, 274, 275…

In [13]:
amap.save_to_html()

Map saved to keplergl_map.html!


In [22]:
m.to_csv('dd_report.csv', index=False)